In [42]:
import feedparser
from newspaper import Article
import html

"""
Every minute;
1. Fetch feed for all RSS URLs. Filter out feed data for last 30 min. - done
2. Maintain a cache with TTL of 60 min. If the current feed title does not exist in the cache, push it and add it to processinglist
4. For each of this title, analyse the news

"""


def get_summary(url):
    article = Article(url)
    article.download()
    article.parse()
    #print("Authors:", article.authors)
    return article.text

def get_articles(url):
    feed = feedparser.parse(url)
    for entry in feed.entries:
        title = entry.title
        #description = entry.summary
        published_date = entry.published
        text = get_summary(entry.links[0]['href'])
        print(f"{published_date} - {html.unescape(title)}")
        #print(f"Published: {published_date}")
        #print(f"Text: {text}")
        print("----------------------------------------------------------------------------------------------------")

get_articles('https://www.business-standard.com/rss/markets-106.rss')
# get_articles('https://rss.app/feeds/tgll9PXJFaUoCjGV.xml')

In [56]:
feed[0].title_detail.base

'asdas'

In [84]:
import requests
import xml.etree.ElementTree as ET

url = "https://www.business-standard.com/markets/capital-market-news/bank-of-maharashtra-allots-61-01-lakh-equity-shares-under-qip-issue-124100500505_1.html"

payload = ""
headers = {
    "User-Agent": "insomnium/0.2.3-a"
}

response = requests.request("GET", url, headers=headers)

#print(response.text)

article = Article('')
article.set_html(response.text)
article.html

'<!DOCTYPE html><html lang="en-US"><head><meta charSet="utf-8"/><link rel="preconnect" href="https://bsmedia.business-standard.com"/><link href="https://securepubads.g.doubleclick.net/tag/js/gpt.js?network-code=6516239" rel="preload" as="script"/><link rel="preload" href="https://bsmedia.business-standard.com/include/_mod/site/html5/images/business-standard-logo.png" as="image"/><link rel="preconnect" href="https://bsmedia.business-standard.com"/><title>Bank of Maharashtra allots 61.01 lakh equity shares under QIP issue | Capital Market News - Business Standard</title><link rel="icon" href="https://www.business-standard.com/favicon.ico"/><link rel="icon" href="https://www.business-standard.com/favicon-16x16.png" sizes="16x16"/><link rel="icon" href="https://www.business-standard.com/favicon-32x32.png" sizes="32x32"/><link rel="icon" href="https://www.business-standard.com/android-chrome-192x192.png" sizes="192x192"/><link rel="apple-touch-icon" href="https://www.business-standard.com/a

In [83]:
# Parse the article
article.parse()

# Now you can access the parsed content
print("Title:", article.title)
print("Text:", article.text)
print("Authors:", article.authors)
print("Published Date:", article.publish_date)

Title: Bank of Maharashtra allots 61.01 lakh equity shares under QIP issue
Text: 
Authors: ['Business Standard']
Published Date: 2024-10-05 17:08:00+05:30


In [61]:
response = requests.request(
    method="GET",
    headers={
        "User-Agent": "insomnium/0.2.3-a"
    },
    url="https://www.business-standard.com/rss/markets-106.rss"
)

if not 200 <= response.status_code < 400:
    raise Exception (f"Failed to fetch feed from RSS {url}")

feedData = feedparser.parse(response.text)

In [88]:
article.html

'<!DOCTYPE html><html lang="en-US"><head><meta charSet="utf-8"/><link rel="preconnect" href="https://bsmedia.business-standard.com"/><link href="https://securepubads.g.doubleclick.net/tag/js/gpt.js?network-code=6516239" rel="preload" as="script"/><link rel="preload" href="https://bsmedia.business-standard.com/include/_mod/site/html5/images/business-standard-logo.png" as="image"/><link rel="preconnect" href="https://bsmedia.business-standard.com"/><title>Bank of Maharashtra allots 61.01 lakh equity shares under QIP issue | Capital Market News - Business Standard</title><link rel="icon" href="https://www.business-standard.com/favicon.ico"/><link rel="icon" href="https://www.business-standard.com/favicon-16x16.png" sizes="16x16"/><link rel="icon" href="https://www.business-standard.com/favicon-32x32.png" sizes="32x32"/><link rel="icon" href="https://www.business-standard.com/android-chrome-192x192.png" sizes="192x192"/><link rel="apple-touch-icon" href="https://www.business-standard.com/a

In [97]:
from bs4 import BeautifulSoup
import json
import requests
import xml.etree.ElementTree as ET

url = "https://www.business-standard.com/markets/capital-market-news/avantel-consolidated-net-profit-rises-42-50-in-the-september-2024-quarter-124100500332_1.html"

payload = ""
headers = {
    "User-Agent": "insomnium/0.2.3-a"
}

response = requests.request("GET", url, headers=headers)


soup = BeautifulSoup(response.text, 'html.parser')

# Example: Extract the text from all <p> tags (adjust according to the HTML structure)

# Find the specific <script> tag containing the desired data
# For example, assuming the data is embedded in a script tag containing "window.articleData" or similar
script_tags = soup.find_all('script', type='application/ld+json')

# Initialize a variable to hold the article body
article_body = None

# Loop through all script tags and look for the NewsArticle
for script_tag in script_tags:
    try:
        # Parse the JSON content
        data = json.loads(script_tag.string)
        
        # Check if it's of type NewsArticle and extract the articleBody
        if data.get('@type') == 'NewsArticle':
            article_body = data.get('articleBody')
            break  # Exit loop after finding the first NewsArticle
    except json.JSONDecodeError:
        continue  # Skip any script that does not contain valid JSON

# Print the extracted article body
if article_body:
    print("Article Body:", article_body)
else:
    print("Article Body not found.")

Article Body: Sales rise 42.50% to Rs 77.42 croreNet profit of Avantel rose 42.50% to Rs 22.90 crore in the quarter ended September 2024 as against Rs 16.07 crore during the previous quarter ended September 2023. Sales rose 42.50% to Rs 77.42 crore in the quarter ended September 2024 as against Rs 54.33 crore during the previous quarter ended September 2023.ParticularsQuarter EndedSep. 2024Sep. 2023% Var.Sales77.4254.33 42 OPM %44.8145.92 -PBDT34.4623.84 45 PBT31.5022.20 42 NP22.9016.07 43 Powered by Capital Market - Live News.storycontent div {margin:10px 0;}
